In [32]:
from forward_backward_AD import *

import numpy as np
import torch
import time, timeit

import scipy.io

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
workspace = scipy.io.loadmat(r'../Data Sets/Debug Data/workspace_calc_grad.mat')

c:\Users\Bruker\anaconda3\envs\SAXSTT\lib\site-packages\scipy\io\matlab\mio.py:226: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [3]:
print(workspace.keys())

dict_keys(['__header__', '__version__', '__globals__', 'opt_inputs', 'p', 's', 'projection', 'E', 'numOfCoeffs', 'skip_optimization', 'None', 'return_synth_proj', 'return_Ereg', 'find_coefficients', 'find_orientation', 'find_grad', 'phi_det', 'theta_det', 'l', 'm', 'nx', 'ny', 'nz', 'numOfsegments', 'numOfvoxels', 'mask3D', 'numOfpixels', 'numOfordersopt', 'theta_struct', 'phi_struct', 'a', 'ii', 'grad_a', 'grad_theta_struct', 'grad_phi_struct', 'zeros_struct', 'Ylm_coef', 'a_temp', 'a_temp1', 'a_temp2', 'ones_struct', 'unit_q_beamline', 'N', 'x', 'y', 'z', 'X', 'Y', 'Z', '__function_workspace__'])


In [4]:
theta_struct_it = workspace['theta_struct']
phi_struct_it = workspace['phi_struct']
a_temp_it =    workspace['a_temp']
ny = workspace['ny']
nx = workspace['nx']
nz = workspace['nz']
numOfsegments = workspace['numOfsegments']
current_projection = workspace['projection'][0,0]#[2]
p = workspace['p']
X = workspace['X']
Y = workspace['Y']
Z = workspace['Z']
numOfpixels = workspace['numOfpixels']
unit_q_beamline = workspace['unit_q_beamline']
Ylm_coef = workspace['Ylm_coef']
find_coefficients = workspace['find_coefficients']
find_orientation = workspace['find_orientation']
numOfCoeffs = workspace['numOfCoeffs']
numOfvoxels = workspace['numOfvoxels']



In [39]:
#print(current_projection[0,0][2].shape)
# Testing shit

print(type(numOfvoxels))

yo = int(np.squeeze(numOfvoxels))

print(yo)

print(type(yo))

new_tens = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16], dtype=torch.float64).reshape(4,4)
print(new_tens)


A = torch.rand((3,3,64))
B = torch.rand((3,8))
C = torch.matmul(A[0], B) # Have to figure out how this works. 


<class 'numpy.ndarray'>
64
<class 'int'>
tensor([[ 1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12.],
        [13., 14., 15., 16.]], dtype=torch.float64)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x64 and 3x8)

In [34]:
main(theta_struct_it, phi_struct_it, a_temp_it, ny, nx, nz, numOfsegments, current_projection, p, X, Y, Z, numOfpixels, unit_q_beamline, Ylm_coef, find_coefficients, find_orientation, numOfCoeffs, numOfvoxels)

DEBUG:root:Rot_str shape: torch.Size([3, 3, 64])
DEBUG:root:unit_q_object shape: torch.Size([3, 8])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (9x64 and 3x8)